In [1]:
import sys
sys.path.insert(0, '../')

# Media Objects

Yuntu has `Media` objects to hold numpy arrays and additional information regarding its location in time-frequency space. Media objects also implement auxiliary functions for storing, reading and representig such objects.

#### Creating a media object

All `media` objects require at construction that the user specifies the following:

* `path` (`str`) \[optional\]
* `array` (`np.ndarray`) \[optional\]
* `lazy` (`bool`) \[optional\]
* `window` (`yuntu.Window`) \[optional\]

Either path or array must be given. If an array and path are simultaneosly defined, the path will be used automatically when calling the `write` method but no loading will occurr.

#### Media object attributes

All `Media` objects have the following attributes:

1. `path`: Indicates the path (remote or in filesystem) from which to load the media array. This path will also
be used as the default path in which to save the media object. Possibly empty.
2. `array`: The media numpy array. Access to this attribute will trigger a loading event (see lazy loading).
3. `window`: A `yuntu.Window` object that specifies the portion of Time-Frequency space that is occupied by this media object.
4. `path_ext`: The extension of the `path`, if given.

#### Lazy loading

If no `array` is provided at intialization, the array will be retrieved from the file pointed to by the provided `path`. This will be done at initialization unless the user specifies `lazy=True`. If `lazy=True` the loading of the array will be delayed until the user tries to access the contents of the array.

To check if a `Media` object has loaded the array into memory without triggering the loading process use:

    media.is_empty()
    
If you wish to free memory by dumping the array contents but keep reference to the media object, call method:

    media.clean()
    
When contents of the array are newly required another loading of the array from path will be triggered.

#### Remote paths

If the user specifies a path in the filesystem, yuntu will try to load the array from the requested path. But if the path provided is a url with `http` or `scp` schemas, yuntu will attempt to download the file to a temporary directory and will later use the `load` method on the downloaded file.

The loading of the remote paths is implemented in the `remote_load` method and is generic, so there should be no reason to modify it. But modify it you wish to implement downloading with a new scheme or to modify the download process. 

The `path_exist` method will return if the current `path` exists in the filesystem, and `is_remote` will compare the path agains known remote url patterns.

#### Media objects as Numpy Arrays

Although `Media` objects are not numpy arrays *per se* they will behave exactly as a `numpy.ndarray`. All methods of a `numpy.ndarray` are available to `Media` objects, and they will operate with other numpy arrays, methods or functions in the usual way.

If you wish to access the raw numpy array within the `Media` objects, use the attribute:

    media.array

#### Simple (JSON) representations of Media Objects

All media objects have a simple "JSON"-able representation accesible through the method:
    
    data = media.to_dict()
    
This information can then be used to reconstruct the media object by with:

    Media.from_dict(data)

#### Defining a new Media object

The `Media` class is abstract, and thus serves as a universal interface for all such objects. Any `Media` object type must implement the following methods:

* `load`
* `write`
* `plot`

If new information is required to construct the media object, you should also consider extending the `to_dict` method.

#### Some extra methods

All media objects can be normalized with:

    normalized = media.normalize()
    
A copy of the media object can be obtained by:

    copy = media.copy()

## Temporal Media Objects

Media objects that have explicit time reference for the data it holds (such as audio/video data, or an spectrogram) should also inherit from the special mixin `TimeMediaMixin`. These require that the user provides information on the temporal range over which the media object holds data. 

It is assumed that a single axis of the `Media` array represents the `time` axis, so that slices along this axis represents the media value at diferent times. By default, the time axis is the 0-th axis. For example, `Audio` objects are `TemporalMedia` objects with the first axis being the time axis. Therefore

    audio[0], audio[1], ..., audio[n], ...
    
represent the audio contents at different times. Also, `Spectrogram` objects are also `TimeMedia` objects (but also `FrequencyMedia` objects) but with the second axis as the time axis. Thus

    spectrogram[:, 0], spectrogram[:, 1], ..., spectrogram[:, n], ...
    
represent spectrogram slices at different times.

#### Creating a temporal media object

All `TemporalMedia` object require, aside from the usual media requirements, the following data:

1. `start` (`float`) [optional]: Time at which the media object starts. Defaults to 0. Useful when doing things around a reference object.
2. `duration` (`float`) [optional]: Temporal length of the media object.
3. `resolution` (`float`) [optional]: The temporal resolution of the temporal sampling. This translates to the number of array bins per second.
4. `time_axis` (`TimeAxis`) [optional]: The temporal axis object that holds information on the mapping of time to array bins.

If no `time_axis` is provided, the `start`, `duration` and `resolution` arguments will be used to create a new `TimeAxis` instance. Otherwise, this information will be discarded as the `time_axis` object has precedence.

#### Time Axis